# Closure-learning

In [1]:
import h5py
import numpy as np
from copy import deepcopy
import os

from library.misc.io import load_fields_from_hdf5
import library.preprocessing.openfoam_moments as openfoam_moments
from tests.simulations.test_smm import test_restart_from_openfoam as smm_restart_from_openfoam
main_dir = os.getenv("SMS")
foam_sim = os.getenv("FOAM_SIM")

## Project Openfoam 3d data into moment format

Settings:

In [2]:
level = 1
filepath_openfoam_case = os.path.join(foam_sim, f'multiphase/interFoam/RAS/channelflow_mid/')
filepath_target_mesh = os.path.join(os.path.join(main_dir, 'meshes/simple_openfoam/mesh_2d_mid.msh'))


Project openfoam (3d) to moments:

In [3]:
openfoam_moments.convert_openfoam_to_moments(filepath_openfoam_case, level, filepath_target_mesh, meshtype_order='triangle')

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200]

Conversion 1/201 completed.
Conversion 2/201 completed.
Conversion 3/201 completed.
Conversion 4/201 co

Generate training dataset

In [17]:
smm_restart_from_openfoam(level=level)

number of available cpus: 8
rm -f *.gch *.o *.dat 
rm -rf /home/ingo/Git/SMM/shallow-moments-simulation/bin
make command : make DIMENSION=2 N_BOUNDARY_CONDITIONS=4 N_ELEMENTS=1210 N_FIELDS=5 N_FIELDS_AUX=0 PATH_SETTINGS=/home/ingo/Git/SMM/shallow-moments-simulation/outputs/output_c/settings.hdf5 PATH_MESH=/home/ingo/Git/SMM/shallow-moments-simulation/outputs/output_c/mesh.hdf5 PATH_FIELDS=/home/ingo/Git/SMM/shallow-moments-simulation/outputs/output_c/fields.hdf5 TIMESTEPPER=Adaptive TIMESTEPPER_PARAM=0.45 ODE_FLUX=1 ODE_SOURCE=-1
-e Building for CPU...
make[1]: Entering directory '/home/ingo/Git/SMM/shallow-moments-simulation/library/solver'
`which mpic++` -I./ -I/home/ingo/Git/SMM/shallow-moments-simulation/dependencies/kokkos/core/src -I/home/ingo/Git/SMM/shallow-moments-simulation/dependencies/kokkos/containers/src -I/home/ingo/Git/SMM/shallow-moments-simulation/dependencies/kokkos/algorithms/src -I/home/ingo/Git/SMM/shallow-moments-simulation/dependencies/kokkos/tpls/desul/include 

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

...

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

Warning: VTK requires 3D points, but 2D points given. Appending 0 third component.

## Train with multi-trajectory SinDy

### Read input/output dataset

In [18]:

path_data_input = f"{filepath_openfoam_case}/closure_dataset_input.hdf5"
path_data_output = f"{filepath_openfoam_case}/closure_dataset_output.hdf5"

In [19]:
with h5py.File(path_data_input, "r") as f:
    n_iterations = len(f.keys())
    Q, Qaux, time = load_fields_from_hdf5(path_data_input, 0)
    n_elements = Q.shape[0]
    n_fields = Q.shape[1]

input = np.zeros((n_iterations, n_elements, n_fields))
output = np.zeros((n_iterations, n_elements, n_fields))
dts = np.zeros((n_iterations, n_elements)) 
for i in range(n_iterations):
    output[i, :, :] = deepcopy(Q)
    Q, Qaux, time = load_fields_from_hdf5(path_data_input, i)
    input[i, :, :] = deepcopy(Q)
    Q, Qaux, dt = load_fields_from_hdf5(path_data_output, i)
    dts[i, :] = dt
input = input.reshape(n_iterations*n_elements, n_fields)
output = output.reshape(n_iterations*n_elements, n_fields)
dts = dts.reshape(n_iterations*n_elements)


go to primitive variables

In [20]:
for i in range(1, n_fields):
    input[:,i] = input[:,i] / input[:,0]

Sanity checks

In [22]:

print(f"number of NAN entries in output: {np.sum(np.isnan(output) > 0)}/{output.shape[0]*output.shape[1]}")
print(f"number of inf entries in output: {np.sum(np.isinf(output) > 0)}/{output.shape[0]*output.shape[1]}")
print(f"number of non-zero entries in output: {np.sum(np.abs(output) > 0)}/{output.shape[0]*output.shape[1]}")

number of NAN entries in output: 0/1203950
number of inf entries in output: 0/1203950
number of non-zero entries in output: 969722/1203950


Use [Multiple trajectories](https://pysindy.readthedocs.io/en/latest/examples/1_feature_overview/example.html#Single-trajectory,-pass-in-collection-times)

### Warning
because we need at least 3 time steps (maybe a custom fd model helps?), I assume a linear relation in time and sample the output as list [IC, IC+0.5*dQ, IC_dQ]. This is clearly an assumption.

In [23]:
x_data = []
T_data = []
for i in range(n_iterations*n_elements):
    x_data.append(np.stack((input[i, :], input[i,:] + 0.5 *output[i, :], input[i,:] + 1.0 *output[i, :])))
    T_data.append( np.array([0, 0.5*dts[i], 1.0 * dts[i]]) )

Test train split

In [24]:
indices = np.linspace(0, len(x_data)-1, len(x_data), dtype=int)
from sklearn.model_selection import train_test_split
I_train, I_test = train_test_split(indices, test_size=0.01, random_state=42)
x_test = [x_data[i] for i in I_test]
T_test = [T_data[i] for i in I_test]
x_train = [x_data[i] for i in I_train]
T_train = [T_data[i] for i in I_train]
print(f"number of training samples: {len(x_train)}")
print(f"number of test samples: {len(x_test)}")

number of training samples: 238382
number of test samples: 2408


### PySindy

In [25]:
import pysindy as ps 

### Train without any constraints

In [26]:
## Create data
# library_poly = ps.PolynomialLibrary(degree=2, include_bias=False)
library_unsym = ps.CustomLibrary(
    library_functions=[lambda h, u, u1, v, v1: abs(u), lambda h, u, u2, v, v1: abs(v)])
    # library_functions=[lambda h, u, v : abs(u), lambda h, u, v: abs(u)])
combined_library = library_unsym
# feature_names = ['h', 'hu', 'hv']
feature_names = ['h', 'hu', 'hu1', 'hv', 'hv1']
model_unsym = ps.SINDy(feature_library = combined_library, feature_names = feature_names)
# model = ps.SINDy(feature_library = library_poly)
model_unsym.fit(x_train, t=T_train, multiple_trajectories=True, unbias=True)
model_unsym.print()
 

(h)' = 1.985 f1(h,hu,hu1,hv,hv1)
(hu)' = 0.189 f0(h,hu,hu1,hv,hv1) + 8.152 f1(h,hu,hu1,hv,hv1)
(hu1)' = 1.103 f1(h,hu,hu1,hv,hv1)
(hv)' = 0.271 f1(h,hu,hu1,hv,hv1)
(hv1)' = 0.000


/home/ingo/Git/SMM/shallow-moments-simulation/.venv/lib/python3.9/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(


In [27]:
def compute_error(model, x_test, T_test):
    # considered_fields = [1, 2]
    considered_fields = [1]
    error = 0.
    rel_error = 0.
    mean = 0.
    n_tests = len(x_test)
    for i in range(n_tests):
        x0 = x_test[i][0]
        y = x_test[i][-1, considered_fields]
        time = T_test[i][:]
        y_sim = model.simulate(x0, time)[-1, considered_fields]
        x0_selected = x0[considered_fields]
        dy_no_closure = y-x0_selected
        dy_with_closure = y_sim-x0_selected
        error += np.linalg.norm((dy_no_closure))
        rel_error+= np.linalg.norm((dy_with_closure)) 
    error = error/n_tests
    rel_error = rel_error/n_tests
    print(f'base L2 error {error}')
    print(f'closure L2 error {rel_error}')
    print(f'closure rel L2 error {rel_error/error}')


Test

In [28]:
print(compute_error(model_unsym, x_test , T_test))
# print(compute_error(model_unsym, x_train, T_train))

base L2 error 0.007542096055960814
closure L2 error 0.0031387581585930902
closure rel L2 error 0.4161652324902978
None


### Constraint problem for level = 0

In [ ]:
## Create data
n_constraints = 5
n_targets = x_train[0].shape[1]
n_features = 2
n_eqns = 3
constraint_rhs = np.zeros(n_constraints)
constraint_lhs = np.zeros((n_constraints, n_eqns*n_features))
# coefficients of Abs(u) == Abs(v) in u'
constraint_lhs[0, n_features + 0] = 1
constraint_lhs[0, n_features + 1] = -1
# coefficients of Abs(u) == Abs(v) in v'
constraint_lhs[1, 2*n_features + 0] = 1
constraint_lhs[1, 2*n_features + 1] = -1
# coefs for u' and v' are the same
constraint_lhs[2, 1*n_features + 0] = 1
constraint_lhs[2, 2*n_features + 0] = -1
# h' is not affcted from Abs(u)
constraint_lhs[3, 0] = 1
# h' is not affcted from Abs(v)
constraint_lhs[4, 1] = 1

opt = ps.ConstrainedSR3(constraint_lhs=constraint_lhs, constraint_rhs=constraint_rhs, threshold=0.5, thresholder='l1')
# opt = ps.SR3(threshold=0.5, thresholder='l1')
library_sym = ps.CustomLibrary(
    library_functions=[lambda x, y, z: abs(y), lambda x, y, z: abs(z)])
    # library_functions=[lambda h, u, u1, v, v1: abs(u), lambda h, u, u2, v, v1: abs(v)])
combined_library = library_sym
feature_names = ['h', 'u', 'v']
# feature_names = ['h', 'u', 'u1', 'v', 'v1']
model_sym = ps.SINDy(feature_library = combined_library, feature_names = feature_names, optimizer=opt)


model_sym.fit(x_train, t=T_train, multiple_trajectories=True)
model_sym.print()
 

In [ ]:
print(compute_error(model_sym, x_test , T_test))
print(compute_error(model_sym, x_train, T_train))

### Constraint problem for level = 1

In [29]:

## Create data
n_constraints = 16
n_targets = x_train[0].shape[1]
n_features = 4
n_eqns = 5
constraint_rhs = np.zeros(n_constraints)
constraint_lhs = np.zeros((n_constraints, n_eqns*n_features))
# coefficients of Abs(u) == Abs(v) in u'
constraint_lhs[0, n_features + 0] = 1
constraint_lhs[0, n_features + 2] = -1
# coefficients of Abs(u) == Abs(v) in v'
constraint_lhs[1, 3*n_features + 0] = 1
constraint_lhs[1, 3*n_features + 2] = -1
# coefs for u' and v' are the same
constraint_lhs[2, 1*n_features + 0] = 1
constraint_lhs[2, 3*n_features + 0] = -1

# coefficients of Abs(u1) == Abs(v1) in u'
constraint_lhs[3, n_features + 1] = 1
constraint_lhs[3, n_features + 3] = -1
# coefficients of Abs(u1) == Abs(v1) in v'
constraint_lhs[4, 3*n_features + 1] = 1
constraint_lhs[4, 3*n_features + 3] = -1
# coefs for u1' and v1' are the same
constraint_lhs[5, 1*n_features + 1] = 1
constraint_lhs[5, 3*n_features + 1] = -1

# coefficients of Abs(u) == Abs(v) in u1'
constraint_lhs[6, 2*n_features + 0] = 1
constraint_lhs[6, 2*n_features + 2] = -1
# coefficients of Abs(u) == Abs(v) in v1'
constraint_lhs[7, 4*n_features + 0] = 1
constraint_lhs[7, 4*n_features + 2] = -1
# coefs for u' and v' are the same
constraint_lhs[8, 2*n_features + 0] = 1
constraint_lhs[8, 4*n_features + 0] = -1
 
# coefficients of Abs(u1) == Abs(v1) in u1'
constraint_lhs[9, 2*n_features + 1] = 1
constraint_lhs[9, 2*n_features + 3] = -1
# coefficients of Abs(u1) == Abs(v1) in v1'
constraint_lhs[10, 4*n_features + 1] = 1
constraint_lhs[10, 4*n_features + 3] = -1
# coefs for u1' and v1' are the same
constraint_lhs[11, 2*n_features + 1] = 1
constraint_lhs[11, 4*n_features + 1] = -1

# h' is not affcted from Abs(u)
constraint_lhs[12, 0] = 1
# h' is not affcted from Abs(v)
constraint_lhs[13, 1] = 1
# h' is not affcted from Abs(u1)
constraint_lhs[14, 2] = 1
# h' is not affcted from Abs(v1)
constraint_lhs[15, 3] = 1

opt = ps.ConstrainedSR3(constraint_lhs=constraint_lhs, constraint_rhs=constraint_rhs, threshold=0.5, thresholder='l1')
# opt = ps.SR3(threshold=0.5, thresholder='l1')
# library_poly = ps.PolynomialLibrary(degree=2, include_bias=False)
library_sym_1 = ps.CustomLibrary(
    # library_functions=[lambda x, y, z: abs(y), lambda x, y, z: abs(z)])
    library_functions=[lambda h, u, u1, v, v1: abs(u), lambda h, u, u1, v, v1: abs(u1), lambda h, u, u1, v, v1: abs(u)**(7/3),  lambda h, u, u1, v, v1: abs(u1)**(7/3)])
combined_library = library_sym_1
# feature_names = ['h', 'hu', 'hv']
feature_names = ['h', 'hu', 'hu1', 'hv', 'hv1']
model_sym_1 = ps.SINDy(feature_library = combined_library, feature_names = feature_names, optimizer=opt)


model_sym_1.fit(x_train, t=T_train, multiple_trajectories=True)
model_sym_1.print()
 

(h)' = 0.000
(hu)' = 0.000
(hu1)' = 0.000
(hv)' = 0.000
(hv1)' = 0.000


In [30]:

print(compute_error(model_sym_1, x_test , T_test))
# print(compute_error(model_sym_1, x_train, T_train))

base L2 error 0.007542096055960814
closure L2 error 0.0
closure rel L2 error 0.0
None


In [31]:
## Create data
n_constraints = 8
n_targets = x_train[0].shape[1]
n_features = 4
n_eqns = 5
constraint_rhs = np.zeros(n_constraints)
constraint_lhs = np.zeros((n_constraints, n_eqns*n_features))
# coefs for u' and v' are the same
constraint_lhs[0, 1*n_features + 0] = 1
constraint_lhs[0, 3*n_features + 0] = -1

# coefs for u1' and v1' are the same
constraint_lhs[1, 1*n_features + 1] = 1
constraint_lhs[1, 3*n_features + 1] = -1

# coefs for u' and v' are the same
constraint_lhs[2, 2*n_features + 0] = 1
constraint_lhs[2, 4*n_features + 0] = -1
 
# coefs for u1' and v1' are the same
constraint_lhs[3, 2*n_features + 1] = 1
constraint_lhs[3, 4*n_features + 1] = -1

# h' is not affcted from Abs(u)
constraint_lhs[4, 0] = 1
# h' is not affcted from Abs(v)
constraint_lhs[5, 1] = 1
# h' is not affcted from Abs(u1)
constraint_lhs[6, 2] = 1
# h' is not affcted from Abs(v1)
constraint_lhs[7, 3] = 1

opt = ps.ConstrainedSR3(constraint_lhs=constraint_lhs, constraint_rhs=constraint_rhs, threshold=0.5, thresholder='l1')
# opt = ps.SR3(threshold=0.5, thresholder='l1')
# library_poly = ps.PolynomialLibrary(degree=2, include_bias=False)
library_sym_1 = ps.CustomLibrary(
    # library_functions=[lambda x, y, z: abs(y), lambda x, y, z: abs(z)])
    library_functions=[lambda h, u, u1, v, v1: abs(u), lambda h, u, u1, v, v1: abs(u1), lambda h, u, u1, v, v1: abs(u)**(7/3),  lambda h, u, u1, v, v1: abs(u1)**(7/3)])
combined_library = library_sym_1
# feature_names = ['h', 'hu', 'hv']
feature_names = ['h', 'hu', 'hu1', 'hv', 'hv1']
model_sym_1 = ps.SINDy(feature_library = combined_library, feature_names = feature_names, optimizer=opt)


model_sym_1.fit(x_train, t=T_train, multiple_trajectories=True)
model_sym_1.print()
 

(h)' = 0.000
(hu)' = 0.291 f0(h,hu,hu1,hv,hv1) + -0.145 f1(h,hu,hu1,hv,hv1)
(hu1)' = 0.000
(hv)' = 0.291 f0(h,hu,hu1,hv,hv1) + -0.145 f1(h,hu,hu1,hv,hv1)
(hv1)' = 0.000


In [32]:
print(compute_error(model_sym_1, x_test , T_test))
# print(compute_error(model_sym_1, x_train, T_train))

base L2 error 0.007542096055960814
closure L2 error 0.0025594488518678967
closure rel L2 error 0.3393551120109461
None


## TODOs
- [ ] Comparison to multistep integration
- [ ] Comparison of error in 3D (different projection)
- [x] check why there are so little non-zeros. Something wrong in callback.h?
- [x] generate mid/high resulition data
- [x] use only newtonian model, not chezy
- [x] use level 0 system.
- [ ] Ideally, I do not want to recompute the moment projection of the high fidality data every time. Can I just use the first N-levels of a given one? I think this requires changes in `callback.h` and fit the Q fields to the correct size and PICK THE RIGHT FIELDS.